In [1]:
from math import factorial
import sympy as sp

In [2]:
def sym_basis_k_symbolic(k, d=3):
    basis = []

    # Factorials and sqrt become symbolic
    factorial = sp.factorial
    sqrt = sp.sqrt

    def generate(remain, parts):
        if len(parts) == d - 1:
            parts = parts + [remain]
            a, b, c = parts

            norm = sqrt(factorial(a) * factorial(b) * factorial(c) / factorial(k))
            basis.append((tuple(parts), sp.simplify(norm)))
            return

        for i in range(remain, -1, -1):
            generate(remain - i, parts + [i])

    generate(k, [])
    return basis


# Test
for vec, norm in sym_basis_k_symbolic(4):
    print(vec, norm)

(4, 0, 0) 1
(3, 1, 0) 1/2
(3, 0, 1) 1/2
(2, 2, 0) sqrt(6)/6
(2, 1, 1) sqrt(3)/6
(2, 0, 2) sqrt(6)/6
(1, 3, 0) 1/2
(1, 2, 1) sqrt(3)/6
(1, 1, 2) sqrt(3)/6
(1, 0, 3) 1/2
(0, 4, 0) 1
(0, 3, 1) 1/2
(0, 2, 2) sqrt(6)/6
(0, 1, 3) 1/2
(0, 0, 4) 1


In [3]:
import sympy as sp
from functools import lru_cache


def pi_symmetric_multinomial_opt(M, basis, numeric=False):
    """
    Compute the symmetric multinomial projection of M on the given basis.

    Args:
        M (sp.Matrix): d x d symbolic matrix.
        basis (list): List of (vector, norm) tuples from sym_basis_k_symbolic.
        numeric (bool): If True, evaluates numerically to speed up large k.

    Returns:
        sp.Matrix: Projected matrix.
    """
    d = M.rows
    n = len(basis)
    piM = sp.zeros(n, n)
    factorial = sp.factorial

    # Maximum degree k (all basis vectors have same degree)
    max_k = sum(basis[0][0])
    factorials = [factorial(i) for i in range(max_k + 1)]

    # Precompute powers of M
    powers = {}
    for u in range(d):
        for v in range(d):
            powers[(u, v)] = [1]  # M[u,v]^0
            val = 1
            for t in range(1, max_k + 1):
                val *= M[u, v]
                powers[(u, v)].append(val)

    # Function to enumerate contingency matrices
    def enum_contingency(rows, cols):
        @lru_cache(maxsize=None)
        def _recurse(row_idx, cols_remaining):
            cols_remaining = list(cols_remaining)
            if row_idx == len(rows):
                if all(c == 0 for c in cols_remaining):
                    return [()]
                return []
            results = []
            target = rows[row_idx]

            def compose(pos, left, current):
                if pos == len(cols_remaining) - 1:
                    val = left
                    if val <= cols_remaining[pos]:
                        yield tuple(current + [val])
                    return
                maxv = min(left, cols_remaining[pos])
                for v in range(maxv, -1, -1):
                    yield from compose(pos + 1, left - v, current + [v])

            for row_choice in compose(0, target, []):
                new_cols = tuple(cols_remaining[j] - row_choice[j] for j in range(len(cols_remaining)))
                for tail in _recurse(row_idx + 1, new_cols):
                    results.append(tuple(row_choice) + tail)
            return results

        return _recurse(0, tuple(cols))

    # Compute projection matrix
    for i_idx, (vec_i, norm_i) in enumerate(basis):
        for j_idx, (vec_j, norm_j) in enumerate(basis):
            val = 0
            for mat_flat in enum_contingency(tuple(vec_i), tuple(vec_j)):
                term = 1
                denom = 1
                for u in range(d):
                    for v in range(d):
                        t = mat_flat[u * d + v]
                        if t:
                            term *= powers[(u, v)][t]
                            denom *= factorials[t]
                val += factorials[max_k] / denom * term
            piM[i_idx, j_idx] = norm_i * norm_j * val
            if numeric:
                piM[i_idx, j_idx] = sp.N(piM[i_idx, j_idx])

    return piM

In [4]:
import sympy as sp
import numpy as np
import pickle
import math
from functools import lru_cache

# --------------------------
# Utilities (from earlier)
# --------------------------
def gen_exponents(k, n=9):
    """Generate exponent tuples of length n summing to k, lexicographic order."""
    def recurse(pos, rem, cur):
        if pos == n - 1:
            yield tuple(cur + [rem])
            return
        for e in range(rem, -1, -1):
            yield from recurse(pos+1, rem-e, cur+[e])
    return list(recurse(0, k, []))

# --------------------------
# Conversion functions
# --------------------------
vars9 = sp.symbols("a b c d e f g h i")

def poly_to_coeff_vector(expr, exps, vars=vars9):
    """
    Convert SymPy polynomial expr into a coefficient vector according to exponent tuples exps.
    Returns numpy 1D float array length = len(exps).
    """
    expr = sp.expand(expr)
    poly = sp.Poly(expr, vars)
    coeffs = np.zeros(len(exps), dtype=float)

    # poly.as_dict gives {exp_tuple: coeff}
    monom_dict = poly.as_dict()
    exp_to_index = {exp: i for i, exp in enumerate(exps)}

    for exp_tuple, coeff in monom_dict.items():
        idx = exp_to_index.get(exp_tuple)
        if idx is None:
            # if expr contains a monomial of degree != k, this is an error for our fixed-degree basis
            raise ValueError(f"Monomial {exp_tuple} not in the chosen degree basis (maybe different total degree).")
        coeffs[idx] = float(coeff)
    return coeffs

def matrix_to_coeff_tensor(mat_sym, k):
    """
    Convert sympy Matrix mat_sym (n x n) whose entries are polynomials of total degree k
    into a numpy array T of shape (n, n, Nmon), where Nmon = C(k+8,8).
    Also returns exps (list of exponent tuples in canonical order).
    """
    exps = gen_exponents(k, 9)
    n = mat_sym.rows
    m = len(exps)
    T = np.zeros((n, n, m), dtype=float)
    for i in range(n):
        for j in range(n):
            T[i,j,:] = poly_to_coeff_vector(mat_sym[i,j], exps)
    return T, exps

import pickle
import numpy as np

def save_coeff_tensor(filename, T, exps):
    """
    Save:
    - T: numeric numpy array (float)
    - exps: list of 9-tuples (Python objects)
    """
    np.save(filename + "_T.npy", T)
    with open(filename + "_exps.pkl", "wb") as f:
        pickle.dump(exps, f, protocol=pickle.HIGHEST_PROTOCOL)




# --------------------------
# Small helpers for testing
# --------------------------
def direct_sympy_eval(mat_sym, A):
    """
    Directly substitute a..i from A into sympy matrix mat_sym and return numpy array result.
    """
    symbols = sp.symbols("a b c d e f g h i")
    subs = {s: float(v) for s, v in zip(symbols, np.array(A).reshape(-1))}
    mat_num = mat_sym.subs(subs)
    # convert to numpy
    return np.array(mat_num.tolist(), dtype=float)

import numpy as np
import pickle

def load_coeff_tensor(filename):
    """
    Load both T and exps with fully preserved structure.
    """
    T = np.load(filename + "_T.npy")
    with open(filename + "_exps.pkl", "rb") as f:
        exps = pickle.load(f)
    return T, exps


def evaluate_tensor_matrix_input(T, exps, A):
    """
    Evaluate coefficient tensor T (n,n,Nmon) using 3x3 matrix A (row-major mapping a..i).
    Returns numeric n x n numpy array.
    """
    # flatten A row-major
    vals = np.array(A).reshape(-1)   # length 9
    exps_arr = np.array(exps)       # shape (Nmon, 9)
    # compute monomials: for each exponent tuple compute prod(vals**exps)
    # vectorized: vals**exps_arr -> (Nmon,9), then prod along axis=1
    # to avoid large intermediate for high exponents, use pow and prod
    monoms = np.prod(np.power(vals, exps_arr), axis=1)  # shape (Nmon,)
    # tensordot contraction over monomial index
    return np.tensordot(T, monoms, axes=([2],[0]))

In [5]:
# --------------------------
# Extended Tests including piM
# --------------------------
def run_tests_with_piM():
    d = 3
    k_val = 5
    # Step 1: symbolic 3x3 matrix
    M_sym = sp.Matrix(d, d, lambda i, j: sp.symbols(chr(97 + i*d + j)))

    # Step 2: generate symmetric basis
    basis = sym_basis_k_symbolic(k_val, d)

    # Step 3: compute π(M) symbolically
    piM_sym = pi_symmetric_multinomial_opt(M_sym, basis, numeric=False)

    # Convert to coefficient tensor and save
    T, exps = matrix_to_coeff_tensor(piM_sym, k_val)
    save_coeff_tensor("test_piM", T, exps)

    # Load back
    T_loaded, exps_loaded = load_coeff_tensor("test_piM")
    piM_eval_loaded = evaluate_tensor_matrix_input(T_loaded, exps_loaded, np.eye(3))  # use identity just to test eval
    piM_eval_orig = evaluate_tensor_matrix_input(T, exps, np.eye(3))

    # Check numeric evaluation equality
    assert np.allclose(piM_eval_loaded, piM_eval_orig, atol=1e-8), "Loaded piM differs from original"

    # Optional: check shapes and basic properties
    assert T.shape == T_loaded.shape, "T shape mismatch after load"
    assert exps == exps_loaded, "Exps mismatch after load"

    print("piM save/load roundtrip OK ✅")
    print("Shape of piM coefficient tensor:", T.shape)
    print("Number of monomials:", len(exps))

# Run the test
run_tests_with_piM()


piM save/load roundtrip OK ✅
Shape of piM coefficient tensor: (21, 21, 1287)
Number of monomials: 1287


In [37]:
import numpy as np
import sympy as sp



# -----------------------------------------------------
# Generate and save π(M) for k = 1..15 in coeff-tensor form
# -----------------------------------------------------

d = 3   # 3×3 matrix

for k_val in range(1, 16):

    print(f"Processing k = {k_val} ...")

    # --------------------------------------------------
    # 1. Create symbolic 3×3 matrix with symbols a..i
    # --------------------------------------------------
    M_sym = sp.Matrix(d, d, lambda i, j: sp.symbols(chr(97 + i * d + j)))

    # --------------------------------------------------
    # 2. Generate the symmetric basis of degree k
    # --------------------------------------------------
    basis = sym_basis_k_symbolic(k_val, d)

    # --------------------------------------------------
    # 3. Compute π(M) symbolically
    # --------------------------------------------------
    piM_sym = pi_symmetric_multinomial_opt(M_sym, basis, numeric=False)

    # --------------------------------------------------
    # 4. Convert π(M) to coefficient tensor + exponent basis
    # --------------------------------------------------
    T, exps = matrix_to_coeff_tensor(piM_sym, k_val)

    # --------------------------------------------------
    # 5. Save to:
    #    piM_sym_k<T>_T.npy
    #    piM_sym_k<T>_exps.pkl
    # --------------------------------------------------
    save_coeff_tensor(f"piM_sym_{k_val}", T, exps)

    print(f"  Saved:  piM_sym_{k_val}_T.npy / piM_sym_{k_val}_exps.pkl")

print("Done.")


Processing k = 1 ...
  Saved:  piM_sym_1_T.npy / piM_sym_1_exps.pkl
Processing k = 2 ...
  Saved:  piM_sym_2_T.npy / piM_sym_2_exps.pkl
Processing k = 3 ...
  Saved:  piM_sym_3_T.npy / piM_sym_3_exps.pkl
Processing k = 4 ...
  Saved:  piM_sym_4_T.npy / piM_sym_4_exps.pkl
Processing k = 5 ...
  Saved:  piM_sym_5_T.npy / piM_sym_5_exps.pkl
Processing k = 6 ...
  Saved:  piM_sym_6_T.npy / piM_sym_6_exps.pkl
Processing k = 7 ...
  Saved:  piM_sym_7_T.npy / piM_sym_7_exps.pkl
Processing k = 8 ...
  Saved:  piM_sym_8_T.npy / piM_sym_8_exps.pkl
Processing k = 9 ...
  Saved:  piM_sym_9_T.npy / piM_sym_9_exps.pkl
Processing k = 10 ...
  Saved:  piM_sym_10_T.npy / piM_sym_10_exps.pkl
Processing k = 11 ...


KeyboardInterrupt: 

In [6]:
import numpy as np
import sympy as sp
from file_handler import save_sparse_coeff_tensor  # new sparse saver

# -----------------------------------------------------
# Generate and save π(M) for k = 1..15 in sparse coeff-tensor form
# -----------------------------------------------------

d = 3   # 3×3 matrix

for k_val in range(1, 11):

    print(f"Processing k = {k_val} ...")

    # --------------------------------------------------
    # 1. Create symbolic 3×3 matrix with symbols a..i
    # --------------------------------------------------
    M_sym = sp.Matrix(d, d, lambda i, j: sp.symbols(chr(97 + i * d + j)))

    # --------------------------------------------------
    # 2. Generate the symmetric basis of degree k
    # --------------------------------------------------
    basis = sym_basis_k_symbolic(k_val, d)

    # --------------------------------------------------
    # 3. Compute π(M) symbolically
    # --------------------------------------------------
    piM_sym = pi_symmetric_multinomial_opt(M_sym, basis, numeric=False)

    # --------------------------------------------------
    # 4. Convert π(M) to dense coefficient tensor + exponent basis
    # --------------------------------------------------
    T_dense, exps = matrix_to_coeff_tensor(piM_sym, k_val)

    # --------------------------------------------------
    # 5. Convert dense tensor to sparse and save
    # --------------------------------------------------
    save_sparse_coeff_tensor(f"piM_sym_{k_val}", T_dense, exps)

    print(f"  Saved sparse: piM_sym_{k_val}_T_sparse.pkl / piM_sym_{k_val}_exps.pkl")

print("Done.")


Processing k = 1 ...
  Saved sparse: piM_sym_1_T_sparse.pkl / piM_sym_1_exps.pkl
Processing k = 2 ...
  Saved sparse: piM_sym_2_T_sparse.pkl / piM_sym_2_exps.pkl
Processing k = 3 ...
  Saved sparse: piM_sym_3_T_sparse.pkl / piM_sym_3_exps.pkl
Processing k = 4 ...
  Saved sparse: piM_sym_4_T_sparse.pkl / piM_sym_4_exps.pkl
Processing k = 5 ...
  Saved sparse: piM_sym_5_T_sparse.pkl / piM_sym_5_exps.pkl
Processing k = 6 ...
  Saved sparse: piM_sym_6_T_sparse.pkl / piM_sym_6_exps.pkl
Processing k = 7 ...
  Saved sparse: piM_sym_7_T_sparse.pkl / piM_sym_7_exps.pkl
Processing k = 8 ...
  Saved sparse: piM_sym_8_T_sparse.pkl / piM_sym_8_exps.pkl
Processing k = 9 ...
  Saved sparse: piM_sym_9_T_sparse.pkl / piM_sym_9_exps.pkl
Processing k = 10 ...
  Saved sparse: piM_sym_10_T_sparse.pkl / piM_sym_10_exps.pkl
Done.
